<a href="https://colab.research.google.com/github/hrushikute/DataAnalytics/blob/master/LSTM_IMDBSubwords8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSTM on IMDB dataset**


## Downaload the data

In [ ]:
import tensorflow_datasets as tfds
# Download the subword encoded pretokenized data set
dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)


In [ ]:
dataset ,info

In [ ]:
# Get the tokenizer
tokenizer = info.features['text'].encoder

In [ ]:
BUFFETR_SIZE = 10000
BATCH_SIZE = 256

# Get traina nd test split
train_data, test_data = dataset['train'],dataset['test']
print(len(train_data), len(test_data))
print(type(train_data))

In [ ]:
# Shuffle the training data
train_dataset = train_data.shuffle(BUFFETR_SIZE)

# Batch and Padd the data

train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

### Build and Compile the model

In [ ]:
import tensorflow as tf
import numpy as np

# Hyper parameters
batch_size=1
timesteps = 20
features = 16
lstm_dim = 8

# lets have array with random values
random_input = np.random.rand(batch_size, timesteps, features)
print(f'Shape of array {random_input.shape}')


# Define LSTM that return single output.
lstm=tf.keras.layers.LSTM(lstm_dim)
result = lstm(random_input)
print(f'Result : {result}')

# Define LSTM that retruns a sequence
lstm_rs = tf.keras.layers.LSTM(lstm_dim,return_sequences=True)
print(f"Lstm  retrun sequences :{lstm_rs}")

In [ ]:
## Implement LSTM stacked architecture.
import tensorflow as tf
##Hyperparametes
embedding_dim=64
lstm_1_dim=64
lstm_2_dim=32
dense_dim=64

# Build the model

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_1_dim, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_2_dim)),
  tf.keras.layers.Dense(dense_dim, activation='relu'),
  tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
) 

## Train the Model 

In [ ]:
NUM_OF_EPOCHS = 3

histroy = model.fit(
    train_dataset,
    epochs=NUM_OF_EPOCHS,
    validation_data=test_dataset
)

Epoch 1/3
98/98 [==============================] - 1898s 19s/step - loss: 0.5298 - accuracy: 0.7379 - val_loss: 0.4188 - val_accuracy: 0.8217
Epoch 2/3
98/98 [==============================] - 1881s 19s/step - loss: 0.3150 - accuracy: 0.8783 - val_loss: 0.3615 - val_accuracy: 0.8486
Epoch 3/3
98/98 [==============================] - 1888s 19s/step - loss: 0.2396 - accuracy: 0.9136 - val_loss: 0.4382 - val_accuracy: 0.8152


In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

# Plot the accuracy and results 
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: ignored